```
- 3500 Test (Ko Shuffle): 75.95%

2. Đánh giá trên bộ 600 câu được tạo bởi 50 cụm mới hoàn toàn: 
- Với BERT cũ: Để bình thường và CLS, SEP bị double => Accuracy: 68.16% (10 epochs). 
Hôm qua (best epoch 90/100 epochs) đánh giá trên data mới toanh này chỉ đạt 46.67%
- Khi Chuyển sang: lowercase và question: {question}. answer: {answer} độ chính xác lên : 69.65% (10 epochs). 

```

---

Dưới đây là bảng so sánh chi tiết giữa **mBERT**, **XLM-RoBERTa**, và **ProphetNet** khi áp dụng vào bài toán phân loại ngữ nghĩa (classification) như bạn mô tả:

---

| **Tiêu chí**                  | **mBERT**                                  | **XLM-RoBERTa**                              | **ProphetNet**                              |
|--------------------------------|--------------------------------------------|---------------------------------------------|---------------------------------------------|
| **Kiến trúc**                 | Transformer                               | Transformer                                 | Transformer + Future N-Gram Prediction     |
| **Dữ liệu huấn luyện**        | Wikipedia (104 ngôn ngữ)                  | CommonCrawl (100 ngôn ngữ, dữ liệu lớn hơn) | Custom Dataset cho Text Generation         |
| **Tối ưu hóa task nào?**      | Phân loại (Classification)                | Phân loại (Classification)                 | Sinh ngữ cảnh tương lai (Text Generation)  |
| **Hiệu suất Classification** | Tốt, phù hợp cho đa ngôn ngữ cơ bản.       | Tốt hơn mBERT nhờ dữ liệu phong phú hơn.   | Không tối ưu cho classification tasks.     |
| **Ngôn ngữ hỗ trợ**           | 104                                       | 100                                         | Tập trung vào tiếng Anh.                   |
| **Kích thước mô hình**        | ~450MB                                    | ~550MB                                      | ~570MB                                      |
| **Khả năng xử lý song ngữ (EN-VI)** | Ổn định nhưng không tốt bằng XLM-RoBERTa. | Tốt hơn nhờ dữ liệu đa dạng.               | Không phù hợp cho đa ngôn ngữ.             |
| **Ưu điểm**                   | Nhẹ, phổ biến, tiết kiệm tài nguyên.      | Hiệu suất cao, tổng quát tốt trên dữ liệu phức tạp. | Sinh ngữ cảnh chính xác, tốt cho QA và summarization. |
| **Nhược điểm**                | Tổng quát kém trên dữ liệu lớn.           | Yêu cầu tài nguyên tính toán cao hơn mBERT. | Không phù hợp cho classification.          |
| **Thời gian inference**       | Nhanh hơn XLM-RoBERTa, ProphetNet.        | Chậm hơn mBERT, nhưng nhanh hơn ProphetNet. | Chậm nhất do cấu trúc dự đoán tương lai.   |
| **Độ chính xác (EN-VI Classification)** | Khá tốt (nhưng dễ bị overfitting).         | Tốt nhất nhờ tính đa dạng dữ liệu.         | Không phù hợp do không tối ưu classification.|

---

### Đánh giá chi tiết:

#### **1. mBERT**:
- **Ưu điểm**: Nhẹ, phổ biến, dễ triển khai. Tiết kiệm tài nguyên.
- **Nhược điểm**: Khả năng tổng quát và độ chính xác kém hơn XLM-RoBERTa, đặc biệt trên tập dữ liệu phức tạp hoặc đa ngôn ngữ (như tiếng Anh-Việt).
- **Khi nào nên dùng**: Nếu tài nguyên tính toán bị giới hạn và task không quá phức tạp.

#### **2. XLM-RoBERTa**:
- **Ưu điểm**: Hiệu suất cao hơn mBERT trong hầu hết các tác vụ đa ngôn ngữ. Đặc biệt tốt cho tiếng Anh và tiếng Việt.
- **Nhược điểm**: Yêu cầu tài nguyên tính toán lớn hơn, tốc độ inference chậm hơn một chút.
- **Khi nào nên dùng**: Nếu ưu tiên độ chính xác cao hơn và tài nguyên không phải vấn đề lớn.

#### **3. ProphetNet**:
- **Ưu điểm**: Tối ưu cho bài toán sinh ngữ cảnh tương lai, như tóm tắt văn bản, trả lời câu hỏi.
- **Nhược điểm**: Không phù hợp với các tác vụ phân loại. Không hỗ trợ tốt đa ngôn ngữ.
- **Khi nào nên dùng**: Nếu bạn làm bài toán text generation hoặc summarization, không phải classification.

---

### **Gợi ý chọn mô hình cho task của bạn**:
1. **Ưu tiên chính xác**: Dùng **XLM-RoBERTa** (hoặc DistilXLM-RoBERTa nếu cần giảm kích thước).
2. **Ưu tiên tốc độ và tài nguyên**: Dùng **mBERT**.
3. **Không dùng ProphetNet**: Vì không tối ưu cho phân loại.

Cần thêm hỗ trợ gì, bạn có thể nói rõ hơn nhé! 😊

### GPT init

```python
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

# 1. Chuẩn bị dữ liệu
def prepare_dataset(file_path, text_columns, label_column, tokenizer, max_seq_length):
    """
    Chuẩn bị dữ liệu từ file Excel.
    - Ghép câu `robot` và `user_answer` thành input
    - Tokenize dữ liệu.
    """
    # Đọc dữ liệu
    df = pd.read_excel(file_path)

    # Ghép văn bản
    def combine_text(row):
        question = row[text_columns[0]].strip().lower()
        answer = row[text_columns[1]].strip().lower() if pd.notna(row[text_columns[1]]) else ""
        return f"question: {question}. answer: {answer}"

    df["input_text"] = df.apply(combine_text, axis=1)

    # Chuyển đổi nhãn thành số
    unique_labels = sorted(df[label_column].unique())
    label2id = {label: idx for idx, label in enumerate(unique_labels)}
    df["label"] = df[label_column].map(label2id)

    # Tokenize
    tokenized_data = tokenizer(
        list(df["input_text"]),
        truncation=True,
        padding=True,
        max_length=max_seq_length
    )
    tokenized_data["labels"] = list(df["label"])

    # Chuyển thành HuggingFace Dataset
    dataset = Dataset.from_dict(tokenized_data)
    return dataset, label2id

# Cấu hình
file_path = "/content/processed_data_example_v4_15000Data.xlsx"  # Đường dẫn file dữ liệu
text_columns = ["robot", "user_answer"]
label_column = "user_intent"
max_seq_length = 128
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Chuẩn bị dataset
train_dataset, label2id = prepare_dataset(file_path, text_columns, label_column, tokenizer, max_seq_length)
print(f"Label mapping: {label2id}")

# 2. Huấn luyện mô hình
num_labels = len(label2id)
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

def compute_metrics(eval_pred):
    """Hàm tính toán các metric."""
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,  # Thay bằng valid_dataset nếu có
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# 3. Đánh giá trên tập test
def evaluate_model(test_file_path):
    """Đánh giá mô hình trên tập test."""
    test_dataset, _ = prepare_dataset(test_file_path, text_columns, label_column, tokenizer, max_seq_length)
    results = trainer.predict(test_dataset)

    print("Metrics:", results.metrics)

    # Xử lý nhãn dự đoán
    predictions = results.predictions.argmax(axis=1)
    test_dataset = test_dataset.to_pandas()
    test_dataset["predicted_label"] = predictions
    test_dataset["predicted_label_name"] = test_dataset["predicted_label"].map({v: k for k, v in label2id.items()})

    # Lưu kết quả ra file
    test_dataset.to_excel("test_results.xlsx", index=False)
    print("Test results saved to test_results.xlsx")

# Đường dẫn tập test
test_file_path = "/content/test1_processed_TEST_500to1000Phrase.xlsx"
evaluate_model(test_file_path)

```

Output: - google colab - tầm 5 min
```
Label mapping: {'intent_fallback': 0, 'intent_learn_more': 1, 'intent_negative': 2, 'intent_neutral': 3, 'intent_positive': 4, 'silence': 5}
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-77-0cf010a58281>:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
 [246/246 07:22, Epoch 1/1]
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.372600	0.351577	0.878297	0.878652
Metrics: {'test_loss': 0.6311067342758179, 'test_accuracy': 0.7761194029850746, 'test_f1': 0.781688838698698, 'test_runtime': 1.7385, 'test_samples_per_second': 346.844, 'test_steps_per_second': 5.752}
Test results saved to test_results.xlsx
```